In [ ]:
pip install pandas google-generativeai


In [ ]:
import pandas as pd
import google.generativeai as genai

# Configure Gemini API
genai.configure(api_key="")

def get_code_difference(actual_solution, generated_solution):
    prompt = f"""
    Compare the following actual and generated Kotlin code for Android Studio.
    Identify the differences and specify which parts of the generated code are correct and which are incorrect.

    **Actual Solution:**
    ```kotlin
    {actual_solution}
    ```

    **Generated Solution:**
    ```kotlin
    {generated_solution}
    ```

    Provide a structured response with:
    - Differences in functionality, logic, or structure.
    - Correct parts of the generated code.
    - Incorrect parts with explanations.
    """

    response = genai.GenerativeModel("gemini-pro").generate_content(prompt)
    return response.text.strip()

# Load dataset from Google Drive
file_path = "/content/model-1 - content.csv"
df = pd.read_csv(file_path)

# Generate differences
df["Difference Analysis"] = df.apply(lambda row: get_code_difference(row["solution"], row["generated_solution"]), axis=1)

# Save updated file
output_path = "/content/code_comparison_with_differences.csv"
df.to_csv(output_path, index=False)

print(f"Updated dataset saved to: {output_path}")


In [ ]:
import pandas as pd
import google.generativeai as genai

def get_code_difference(actual_solution, generated_solution):
    prompt = f"""
    Compare the following actual and generated Kotlin code for Android Studio.
    Identify the differences and specify which parts of the generated code are correct and which are incorrect.

    **Actual Solution:**
    ```kotlin
    {actual_solution}
    ```

    **Generated Solution:**
    ```kotlin
    {generated_solution}
    ```

    Provide a structured response with:
    - Differences in functionality, logic, or structure.
    - Correct parts of the generated code.
    - Incorrect parts with explanations.
    """

    response = genai.GenerativeModel("gemini-pro").generate_content(prompt)
    return response.text.strip()

# Load dataset from Google Drive
file_path = "/content/model-2.csv"
df = pd.read_csv(file_path)

# Generate differences
df["Difference Analysis"] = df.apply(lambda row: get_code_difference(row["solution"], row["generated_solution"]), axis=1)

# Save updated file
output_path = "/content/code_comparison_with_differences2.csv"
df.to_csv(output_path, index=False)

print(f"Updated dataset saved to: {output_path}")

Updated dataset saved to: /content/code_comparison_with_differences2.csv


In [ ]:
import google.generativeai as genai
import pandas as pd

def get_model_analysis(differences):
    """Summarizes model weaknesses, strengths, and areas of improvement with accuracy percentages."""
    prompt = f"""
    Based on the following model-generated code analysis, summarize:
    1. The key weaknesses of the model.
    2. Areas where the model should improve.
    3. Exceptional results achieved by the model.

    **Analysis Data:**
    {differences}

    Also, create a structured table evaluating the model’s performance in different programming concepts.
    Provide accuracy percentages (0-100%) instead of qualitative ratings:

    | Concept                  | Accuracy (%) | Comments |
    |--------------------------|-------------|----------|
    | Logic Implementation     |             |          |
    | Object-Oriented Design   |             |          |
    | Library/API Understanding|             |          |
    | Performance Optimization |             |          |

    Ensure the accuracy values are justified based on the analysis.
    """

    response = genai.GenerativeModel("gemini-pro").generate_content(prompt)
    return response.text.strip()

# Load dataset from Google Drive
file_path = "/content/code_comparison_with_differences1.csv"
df = pd.read_csv(file_path)

# Summarize model performance
analysis_summary = get_model_analysis("\n".join(df["Difference Analysis"].tolist()))

# Save the summary to Drive
summary_path = "/content/model_performance_summary.txt"
with open(summary_path, "w") as file:
    file.write(analysis_summary)

print(f"Model analysis summary saved to: {summary_path}")


In [ ]:
import google.generativeai as genai
import pandas as pd

# Configure Gemini API
genai.configure(api_key="")

def get_model_analysis(differences):
    """Summarizes model weaknesses, strengths, and areas of improvement with accuracy percentages."""
    prompt = f"""
    Based on the following model-generated code analysis, summarize:
    1. The key weaknesses of the model.
    2. Areas where the model should improve.
    3. Exceptional results achieved by the model.

    **Analysis Data:**
    {differences}

    Also, create a structured table evaluating the model’s performance in different programming concepts.
    Provide accuracy percentages (0-100%) instead of qualitative ratings:

    | Concept                  | Accuracy (%) | Comments |
    |--------------------------|-------------|----------|
    | Logic Implementation     |             |          |
    | Object-Oriented Design   |             |          |
    | Library/API Understanding|             |          |
    | Performance Optimization |             |          |

    Ensure the accuracy values are justified based on the analysis.
    """

    response = genai.GenerativeModel("gemini-pro").generate_content(prompt)
    return response.text.strip()

# Load dataset from Google Drive
file_path = "/content/code_comparison_with_differences2.csv"
df = pd.read_csv(file_path)

# Summarize model performance
analysis_summary = get_model_analysis("\n".join(df["Difference Analysis"].tolist()))

# Save the summary to Drive
summary_path = "/content/model_performance_summary2.txt"
with open(summary_path, "w") as file:
    file.write(analysis_summary)

print(f"Model analysis summary saved to: {summary_path}")

Model analysis summary saved to: /content/model_performance_summary2.txt


In [ ]:
import time
import pandas as pd
import google.generativeai as genai
from collections import Counter

# Dictionary to store problem classifications (avoids re-querying Gemini)
problem_concept_map = {}

def categorize_problem(problem_statement):
    """Classifies the problem into different programming concepts and stores the result."""
    if problem_statement in problem_concept_map:
        return problem_concept_map[problem_statement]  # Use cached result

    prompt = f"""
    Analyze the following Android Studio problem statement and classify it into relevant programming concepts.
    The concepts should be from the following categories:
    - Logic Implementation
    - Object-Oriented Design
    - Library/API Understanding
    - Performance Optimization

    **Problem Statement:**
    {problem_statement}

    Provide a structured output with relevant concepts listed as a comma-separated list.
    """

    try:
        response = genai.GenerativeModel("gemini-pro").generate_content(prompt)
        categorized_concepts = response.text.strip()

        # Store result to prevent redundant API calls
        problem_concept_map[problem_statement] = categorized_concepts
        time.sleep(2)  # Avoid API rate limiting

        return categorized_concepts

    except Exception as e:
        print(f"Error processing problem: {problem_statement[:50]}... | Error: {e}")
        return "Unknown"  # Handle API failures gracefully

# Load dataset from Google Drive
file_path = "/content/code_comparison_with_differences.csv"
df = pd.read_csv(file_path)

# Process problems in batches to prevent API exhaustion
batch_size = 5  # Adjust batch size based on API limits
for i in range(0, len(df), batch_size):
    df.loc[i:i+batch_size-1, "Concepts"] = df.loc[i:i+batch_size-1, "problem"].apply(categorize_problem)
    time.sleep(5)  # Extra delay after processing a batch

# Count occurrences of each concept
concept_counter = Counter()
for concepts in df["Concepts"]:
    for concept in concepts.split(", "):
        concept_counter[concept.strip()] += 1

# Print concept distribution
print("\nConcept Distribution in Evaluation Dataset:")
print("-----------------------------------------")
for concept, count in sorted(concept_counter.items(), key=lambda x: x[1], reverse=True):
    print(f"{concept}: {count}")



Concept Distribution in Evaluation Dataset:
-----------------------------------------
Read: 2
Update: 2
Object-Oriented Design: 2
Coroutines: 2
- **Logic Implementation**: Converting temperature from Kelvin to Celsius: 1
parsing Unix timestamp to time string: 1
handling wind speed units
- **Library/API Understanding**: None
- **Performance Optimization**: None
- **Object-Oriented Design**: Encapsulation of weather formatting functions into a utility class: 1
**Object-Oriented Design**: Dagger Hilt: 1
SingletonComponent: 1
Dependency Injection
**Library/API Understanding**: Dagger Hilt: 1
Firebase Firestore: 1
CollectionReference: 1
NotesFirebaseRepository: 1
UseCases
**Logic Implementation**: Use case implementation for CRUD operations (AddNote: 1
DeleteNote: 1
GetNote): 1
- **Logic Implementation:** Location tracking: 1
Dependency injection
- **Object-Oriented Design:** Singleton: 1
Dependency injection: 1
Interface implementation
- **Library/API Understanding:** Dagger Hilt: 1
Fused

In [ ]:
import re
import google.generativeai as genai
import pandas as pd
from collections import Counter

# Path to the stored text file
file_path = "/content/mid.txt"

# Read file content
with open(file_path, "r") as file:
    lines = file.readlines()

# Define categories
categories = {
    "Logic Implementation": [],
    "Object-Oriented Design": [],
    "Library/API Understanding": [],
    "Performance Optimization": [],
}

# Process lines and extract concepts
current_category = None
for line in lines:
    line = line.strip()

    # Detect category headers
    match = re.match(r"-?\s*\*\*(.*?)\*\*:", line)  # Matches "**Category Name:**"
    if match:
        current_category = match.group(1).strip()
        continue  # Move to next line

    # If a valid category, store associated concepts
    if current_category in categories and line:
        concepts = re.split(r"[:,\-\(\)]", line)  # Split by common separators
        for concept in concepts:
            concept = concept.strip()
            if concept and concept.lower() != "none" and not concept.isdigit():
                categories[current_category].append(concept)

# Count occurrences of each concept
concept_counts = Counter()
for concepts in categories.values():
    concept_counts.update(concepts)

# Total count of all concepts
total_count = sum(concept_counts.values())

# Create tabular data
table_data = [
    {"Concept": concept, "Frequency": count, "Percentage": f"{(count / total_count) * 100:.2f}%"}
    for concept, count in concept_counts.most_common()
]

# Convert to DataFrame for pretty printing
df = pd.DataFrame(table_data)

# Gemini prompt to format the table
prompt = f"""
Format the following concept distribution data into a well-structured table.

### **Concept Distribution Table**
{df.to_string(index=False)}

Ensure proper alignment of columns.
"""

# Call Gemini API
try:
    response = genai.GenerativeModel("gemini-pro").generate_content(prompt)
    print("\n**Formatted Concept Distribution Table:**\n")
    print(response.text.strip())  # Print Gemini's formatted table

except Exception as e:
    print(f"Error processing with Gemini: {e}")



**Formatted Concept Distribution Table:**

| Concept | Frequency | Percentage |
|---|---|---|
| **Oriented Design** | 33 | 9.32% |
| **Object** | 19 | 5.37% |
| **Library/API Understanding** | 15 | 4.24% |
| **Logic Implementation** | 12 | 3.39% |
| **Performance Optimization** | 12 | 3.39% |
| Dependency Injection | 7 | 1.98% |
| Logic Implementation | 7 | 1.98% |
| Object | 7 | 1.98% |
| Dependency injection | 5 | 1.41% |
| Library/API Understanding | 5 | 1.41% |
| Performance Optimization | 5 | 1.41% |
| Firebase Firestore | 4 | 1.13% |
| Parcelable | 4 | 1.13% |
| **Not applicable** | 3 | 0.85% |
| Properties | 3 | 0.85% |
| Asynchronous Programming | 3 | 0.85% |
| Encapsulation | 3 | 0.85% |
| Dagger Hilt | 3 | 0.85% |
| Flow | 3 | 0.85% |
| N/A | 3 | 0.85% |
| Sealed Class | 3 | 0.85% |
| Data Class | 3 | 0.85% |
| Interface Implementation | 2 | 0.56% |
| Error handling | 2 | 0.56% |
| assumed | 2 | 0.56% |
| SingletonComponent | 2 | 0.56% |
| Abstraction | 2 | 0.56% |
| Class |

In [ ]:
import pandas as pd
import google.generativeai as genai
import time

# Load the dataset
df = pd.read_csv('/content/code_comparison_with_differences1.csv')

# Function to query Gemini for classification
def analyze_with_gemini(description):
    well_performed, poorly_performed = "", ""

    if isinstance(description, str):
        prompt = f"""
        Given the following code difference analysis:

        "{description}"

        Extract and categorize the information into:
        - Well-Performed parts (where the model did exceptionally well)
        - Poorly-Performed parts (where the model made significant mistakes)

        Respond in the format:
        Well_Performed: <list of well-performed parts>
        Poorly_Performed: <list of poorly performed parts>
        """

        try:
            model = genai.GenerativeModel("gemini-2.0-flash")  # Use a valid model
            response = model.generate_content(prompt)
            result = response.text.strip()

            # Extracting well-performed and poorly-performed parts
            if "Well_Performed:" in result and "Poorly_Performed:" in result:
                parts = result.split("Poorly_Performed:")
                well_performed = parts[0].replace("Well_Performed:", "").strip()
                poorly_performed = parts[1].strip()
        except Exception as e:
            print(f"Error processing entry: {e}")
            time.sleep(2)  # Adding a delay in case of rate limits

    return well_performed, poorly_performed

# Apply the function using Gemini AI
df[['Well_Performed', 'Poorly_Performed']] = df['Difference Analysis'].apply(lambda x: pd.Series(analyze_with_gemini(x)))

# Save to CSV
output_file = "model_performance_analysis.csv"
df[['Well_Performed', 'Poorly_Performed']].to_csv(output_file, index=False)

print(f"Filtered results saved to {output_file}")


In [ ]:
import google.generativeai as genai

genai.configure(api_key="")

models = genai.list_models()
for model in models:
    print(model.name)


models/chat-bison-001
models/text-bison-001
models/embedding-gecko-001
models/gemini-1.0-pro-vision-latest
models/gemini-pro-vision
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-001
models/gemini-1.5-pro-002
models/gemini-1.5-pro
models/gemini-1.5-flash-latest
models/gemini-1.5-flash-001
models/gemini-1.5-flash-001-tuning
models/gemini-1.5-flash
models/gemini-1.5-flash-002
models/gemini-1.5-flash-8b
models/gemini-1.5-flash-8b-001
models/gemini-1.5-flash-8b-latest
models/gemini-1.5-flash-8b-exp-0827
models/gemini-1.5-flash-8b-exp-0924
models/gemini-2.0-flash-exp
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-lite-001
models/gemini-2.0-flash-lite
models/gemini-2.0-flash-lite-preview-02-05
models/gemini-2.0-flash-lite-preview
models/gemini-2.0-pro-exp
models/gemini-2.0-pro-exp-02-05
models/gemini-exp-1206
models/gemini-2.0-flash-thinking-exp-01-21
models/gemini-2.0-flash-thinking-exp
models/gemini-2.0-flash-thinking-exp-1219
models/learnlm-1.5-pro-experim

In [ ]:
import pandas as pd
import google.generativeai as genai
from tqdm import tqdm
import os

# --- SETUP GEMINI API ---
genai.configure(api_key=api)
model = genai.GenerativeModel("gemini-2.0-flash")

# --- LOAD INPUT DATASET ---
input_path = "/content/code_comparison_with_differences_gemini.csv"
df = pd.read_csv(input_path).dropna(subset=["problem"]).copy()

# --- LOAD EXISTING OUTPUT IF EXISTS ---
output_path = "generated_kotlin_solutions.csv"
if os.path.exists(output_path):
    df_existing = pd.read_csv(output_path)
    df = df.merge(df_existing[["problem", "generated_solution"]], on="problem", how="left", suffixes=('', '_existing'))
    df["generated_solution"] = df["generated_solution"].combine_first(df["generated_solution_existing"])
    df.drop(columns=["generated_solution_existing"], inplace=True)
else:
    df["generated_solution"] = None  # Initialize column

# --- ASK USER FOR START INDEX ---
start_index = int(input("🔢 Enter the starting index for generation: "))

# --- GENERATION FUNCTION ---
def generate_kotlin_code(problem):
    try:
        prompt = f"""You are an expert Kotlin Android developer.

The following is a problem description written as comments. Your task is to generate ONLY the working Kotlin code for Android based on it.
Do NOT include any explanations, comments, or any extra text. Output ONLY the Kotlin code.

Problem (in comments):
{problem}

Kotlin Code:"""
        response = model.generate_content(prompt)
        return response.text.strip()
    except Exception as e:
        return f"Error: {str(e)}"

# --- GENERATE ONLY MISSING ENTRIES FROM START INDEX ---
tqdm.pandas()
filtered_df = df[df["generated_solution"].isna()]
start_from_idx = filtered_df.index[filtered_df.index >= start_index]

for idx in tqdm(start_from_idx):
    problem = df.loc[idx, "problem"]
    kotlin_code = generate_kotlin_code(problem)
    df.at[idx, "generated_solution"] = kotlin_code

    # Save progressively without solution column (to reduce file size during intermediate saves)
    df[["problem", "generated_solution"]].to_csv(output_path, index=False)

# --- FINAL SAVE WITH SOLUTION COLUMN ONCE ---
final_df = df[["problem", "solution", "generated_solution"]]
final_df.to_csv(output_path, index=False)

print("✅ All done! File saved as 'generated_kotlin_solutions.csv'")


🔢 Enter the starting index for generation: 41


0it [00:00, ?it/s]

✅ All done! File saved as 'generated_kotlin_solutions.csv'


In [ ]:
import pandas as pd
import google.generativeai as genai
import os
import time

# --- SETUP GEMINI API ---
genai.configure(api_key=api)
model = genai.GenerativeModel("gemini-1.5-pro-latest")

# --- LOAD CSV ---
file_path = "/content/generated_kotlin_solutions.csv"
df = pd.read_csv(file_path)

# If difference analysis file already exists, merge it
output_path = "/content/code_comparison_final.csv"
if os.path.exists(output_path):
    existing_df = pd.read_csv(output_path)
    df = df.merge(existing_df[["problem", "Difference Analysis"]], on="problem", how="left")
else:
    df["Difference Analysis"] = None  # initialize column

# --- USER INPUT: SELECT A SINGLE ROW ---
row_number = int(input("🔢 Enter the row number to analyze: "))

# Skip if already analyzed
if pd.notna(df.loc[row_number, "Difference Analysis"]):
    print("✅ This row already has a difference analysis.")
else:
    actual = df.loc[row_number, "solution"]
    generated = df.loc[row_number, "generated_solution"]

    prompt = f"""
    Compare the following actual and generated Kotlin code for Android Studio.
    Identify the differences and specify which parts of the generated code are correct and which are incorrect.

    **Actual Solution:**
    ```kotlin
    {actual}
    ```

    **Generated Solution:**
    ```kotlin
    {generated}
    ```

    Provide a structured response with:
    - Differences in functionality, logic, or structure.
    - Correct parts of the generated code.
    - Incorrect parts with explanations.
    """

    try:
        response = model.generate_content(prompt)
        df.at[row_number, "Difference Analysis"] = response.text.strip()
        df.to_csv(output_path, index=False)
        print(f"✅ Analysis done and saved for row {row_number}.")
    except Exception as e:
        print(f"❌ Error at row {row_number}: {str(e)}")

    time.sleep(1.5)  # Optional: Rate limit buffer


🔢 Enter the row number to analyze: 40
✅ Analysis done and saved for row 40.


In [ ]:
import google.generativeai as genai
import pandas as pd

# Configure Gemini API
genai.configure(api_key=api)

def get_model_analysis(differences):
    """Summarizes model weaknesses, strengths, and areas of improvement with accuracy percentages."""
    prompt = f"""
    Based on the following model-generated code analysis, summarize:
    1. The key weaknesses of the model.
    2. Areas where the model should improve.
    3. Exceptional results achieved by the model.

    **Analysis Data:**
    {differences}

    Also, create a structured table evaluating the model’s performance in different programming concepts.
    Provide accuracy percentages (0-100%) instead of qualitative ratings:

    | Concept                  | Accuracy (%) | Comments |
    |--------------------------|-------------|----------|
    | Logic Implementation     |             |          |
    | Object-Oriented Design   |             |          |
    | Library/API Understanding|             |          |
    | Performance Optimization |             |          |

    Ensure the accuracy values are justified based on the analysis.
    """

    response = genai.GenerativeModel("gemini-1.5-pro-latest").generate_content(prompt)
    return response.text.strip()

# Load dataset from Google Drive
file_path = "/content/code_comparison_final.csv"
df = pd.read_csv(file_path)

# Summarize model performance
analysis_summary = get_model_analysis("\n".join(df["Difference Analysis"].tolist()))

# Save the summary to Drive
summary_path = "/content/code_comparison_result_gemini.csv"
with open(summary_path, "w") as file:
    file.write(analysis_summary)

# Print the summary
print("===== Model Analysis Summary =====")
print(analysis_summary)
print(f"\nModel analysis summary saved to: {summary_path}")


===== Model Analysis Summary =====
## Summary of Model Weaknesses

The model struggles most consistently with maintaining project-specific context, particularly regarding package structures and import statements.  It often generates code that assumes a standard project layout or makes incorrect assumptions about the locations of classes and interfaces.  This leads to compilation errors or incorrect dependencies.  Another area of weakness is error handling.  While the model sometimes generates basic error checking, it often lacks the robustness and detail of the actual solutions, potentially missing edge cases or failing to provide informative error messages.  Occasionally, the model includes redundant code or unnecessary elements, showing an incomplete grasp of optimal Kotlin idioms.

## Areas for Model Improvement

1. **Context Awareness:** The model needs to be more aware of the project's structure and dependencies.  This includes understanding package organization, import paths, and